In [1]:
"""
General workflow for importing a new session
"""
subject = 'P01' # TODO: change this for each subject
verbose = True  # change this for debugging

import matplotlib
%matplotlib inline

from pipeline import Pipeline
settings = dict(debug=False, mne_log_level='Info', sfreq=64) # optional pipeline settings
pipeline = Pipeline(subject, settings)

debugging !??


In [2]:
pipeline.load_raw(verbose=verbose)
print('done')

Opening raw data file /Users/ahnaaf/Documents/tks/EEG-Shazam/eeg/preprocessing/notebooks/mne/P01-raw.fif...
Isotrak not found
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
done


In [4]:
 
#pipeline.raw.plot_sensors()
#pipeline.raw.plot()

# Reject Bad Channels

In [5]:
# switch to interactive GUI mode to scroll through data
%matplotlib tk 
pipeline.plot_raw();
pass

scroll using cursor keys, click on channels to mark as "bad"


In [ ]:
# switch back to inline mode
%matplotlib inline

In [ ]:
# pipeline.reset_bad_channels() # use this to reset the channel list if needed

In [ ]:
# TODO: change these value manually
# If you would rather apply the bandpass filter first, continue until it is applied 
# and then come back to this cell
#pipeline.mark_bad_channels(None, save_to_raw=False) # nothing to change, None will keep old values
pipeline.mark_bad_channels(['P8', 'P10', 'T8'], save_to_raw=True)
print('done')

In [ ]:
pipeline.print_bad_channels()

In [ ]:
#Doesn't work
pipeline.interpolate_bad_channels() # Note: this will overwrite data

# Check and Merge Trials

In [ ]:
pipeline.check_trial_events()

In [ ]:
pipeline.check_trial_audio_onset_merge(use_audio_onsets=True, verbose=None)

In [ ]:
pipeline.merge_trial_and_audio_onsets()

# Check PSD and Channels

In [ ]:
# pipeline.check_psd(fmax=512)
# pipeline.check_psd(fmax=35)
pipeline.check_psd(fmax=256)
pipeline.alt_check_psd()
# line noise will probably be visible @ multiples of 60Hz

In [ ]:
pipeline.check_channel(0)
# quite some drift and movement in the breaks

# Bandpass Filtering

In [ ]:
pipeline.bandpass_filter()

In [ ]:
%matplotlib tk 
pipeline.plot_raw();
pass

In [ ]:
%matplotlib inline

In [ ]:
pipeline.check_channel(0)
# looks like we got rid of that drift - nice!
# what's that? eyeblinks?

In [ ]:
## check PSD again - after bandpass, before down-sampling
pipeline.alt_check_psd()
pipeline.check_psd(fmax=35)
# 60Hz is still visible

# Beat and EOG Epoching

In [ ]:
#Doesn't work
pipeline.generate_beat_events() # Note: this includes cue-beats !!!
pipeline.beat_epochs.average().plot();

In [ ]:
#Can't test until generate beat events
pipeline.find_eog_events()
# wow - that's a lot of blinking

# Down-Sampling

In [ ]:
#Can't test until generate beat events
pipeline.downsample()

In [ ]:
## PSD after down-sampling:
pipeline.check_psd(fmax=32)
# looks less smooth than without down-sampling

In [ ]:
#Can't test until generate beat events
# check events after after resampling -> should get the same result as above
pipeline.check_resampled_trial_events(plot=True, verbose=False)

# ICA

Can't test anything yet

In [ ]:
pipeline.compute_ica(verbose=True)

In [ ]:
%matplotlib inline
pipeline.plot_ica_components() # static plot

In [ ]:
pipeline.inspect_source_psd(1) # plot the PSD of an IC source to look for alpha activity
pipeline.inspect_source_psd(2)

In [ ]:
pipeline.find_eog_artifact_sources(plot=True, verbose=False)

In [ ]:
pipeline.auto_detect_artifact_components()

In [ ]:
pipeline.select_artifact_sources(selection='auto')

In [ ]:
pipeline.exclude_ica_components(selection=[0, 1, 3,11]) # TODO: adapt selection list
# use cells below to decide, which components to reject, then come back to this cell

In [ ]:
pipeline.inspect_ica_component(11, [0, 700])
pipeline.inspect_source_epochs(11, mode='beats', vmax=5)
pipeline.inspect_source_epochs(11, mode='eog', vmax=5)

In [ ]:
for ic in pipeline.suggested_artifact_components:
# for ic in pipeline.ica.exclude:
    pipeline.inspect_ica_component(ic, [0, 700])
    pipeline.inspect_source_epochs(ic, mode='beats', vmax=5)
    pipeline.inspect_source_epochs(ic, mode='eog', vmax=5)

In [ ]:
# enable this for interactive mode
# %matplotlib tk

# pipeline.plot_sources(mode='raw')   # this may take a while
# pipeline.plot_sources(mode='beats')
pipeline.plot_sources(mode='eog');

In [ ]:
# switch back to non-interactive inline mode
%matplotlib inline

In [ ]:
pipeline.assess_unmixing_quality(verbose=False)

In [ ]:
pipeline.save_ica('100p_64c') # save for later

# Application Test

In [ ]:
pipeline.load_ica('100p_64c')

In [ ]:
raw = pipeline.ica.apply(pipeline.raw, exclude=(pipeline.ica.exclude), copy=False)
if len(raw.info['bads']) > 0:
    raw.interpolate_bads_eeg() # interpolate bad channels afterwards as they are not processed by the ICA

%matplotlib tk 
pipeline.plot_raw();
pass